# UNIVA-Bllossom/DeepSeek-qwen-Bllossom-32B

In [1]:
import torch
import datetime
import os
import time
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, BitsAndBytesConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
# AutoModel

In [3]:
class LLMTester:
    def __init__(self, model_name="./model/LLM/EEVE"):
        self.model_name = model_name
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.tokenizer = None
        self.model = None

    def load_model(self) :
        try :
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )

            self.tokenizer = AutoTokenizer.from_pretrained(
                self.model_name,
                trust_remote_code=True
            )

            # device_map = "auto" if self.device=="cuda" else None
            # device_map = {"": 2} if self.device == "cuda" else None

            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device=="cuda" else torch.float32,
                # device_map=device_map,
                trust_remote_code=True,
                low_cpu_mem_usage=True,
                quantization_config=quantization_config
            )

            if self.tokenizer.pad_token is None :
                self.tokenizer.pad_token = self.tokenizer.eos_token

            return True

        except Exception as e :
            print(f"오류 발생 : {e}")
            return False
        
    def get_test_questions(self) :
        questions = [
            "한 의료진이 환자의 증상을 보고 '기침, 발열, 호흡곤란'이라고 기록했습니다. 이 환자가 최근 해외여행을 다녀왔고, 혈액검사에서 백혈구 수치가 높게 나왔다면, 어떤 추가 검사를 우선적으로 실시해야 하며, 그 이유는 무엇인가요? 또한 이 과정에서 의료진이 고려해야 할 감염관리 프로토콜은 무엇인가요?",
            
            "금융 시장에서 A 기업의 주가가 급락하고, 같은 업종의 B, C 기업 주가도 동반 하락했습니다. 그런데 D 기업만 상승했다면, 이런 현상의 원인을 어떻게 분석해야 하며, 투자자는 어떤 정보를 추가로 수집해서 투자 판단을 내려야 할까요?",
            
            "법률 상담에서 임차인이 '집주인이 갑자기 전세 계약 해지를 통보했고, 이사할 돈이 없다'고 상담을 요청했습니다. 이때 변호사는 어떤 순서로 사실관계를 파악해야 하며, 임차인의 권리를 보호하기 위해 어떤 법적 조치들을 단계적으로 검토해야 할까요?",
            
            "교육 현장에서 한 학생이 수학 성적은 우수하지만 국어 성적이 급격히 떨어졌고, 최근 수업 참여도도 낮아졌습니다. 학부모는 사교육을 늘리자고 하는데, 교사는 어떤 관점에서 이 상황을 분석하고 어떤 해결책을 단계적으로 제시해야 할까요?",
            
            "IT 시스템에서 특정 서버의 CPU 사용률이 90%를 넘고, 응답 시간이 평소보다 3배 느려졌습니다. 동시에 데이터베이스 연결 수도 임계치에 도달했다면, 시스템 관리자는 어떤 순서로 문제를 진단하고 해결해야 하며, 근본적인 원인을 찾기 위해 어떤 로그들을 분석해야 할까요?",
            
            "마케팅 담당자가 온라인 광고 캠페인을 분석하니 클릭률은 높은데 구매 전환율이 낮습니다. 그런데 모바일에서는 전환율이 더 낮고, 특정 연령대에서만 이런 현상이 두드러집니다. 이 데이터를 바탕으로 어떤 가설을 세우고 어떤 개선 방안을 단계적으로 실행해야 할까요?",
            
            "건축 현장에서 콘크리트 타설 후 균열이 발견되었고, 날씨는 평소보다 추웠으며, 사용된 콘크리트의 배합비가 기존과 달랐습니다. 구조 엔지니어는 어떤 방식으로 이 문제의 원인을 규명하고, 안전성을 확보하기 위해 어떤 조치들을 순차적으로 취해야 할까요?",
            
            "심리 상담에서 내담자가 '직장에서 스트레스를 받아 잠을 못 자고, 가족과도 자주 다툰다'고 호소합니다. 그런데 대화 중 어린 시절 이야기를 할 때 표정이 어두워집니다. 상담사는 어떤 방식으로 이 정보들을 연결하여 상담 방향을 설정하고 개입 전략을 수립해야 할까요?",
            
            "레스토랑 운영에서 최근 매출이 20% 감소했는데, 근처에 새로운 경쟁업체가 생겼고, 주요 메뉴의 원재료 가격도 올랐습니다. 고객 리뷰에서는 서비스 지연에 대한 불만이 증가했습니다. 경영자는 이 복합적인 문제를 어떻게 분석하고 우선순위를 정해 해결책을 실행해야 할까요?",
            
            "환경 컨설팅에서 한 공장의 폐수 처리 시설에서 처리 효율이 갑자기 떨어졌고, 미생물 활성도도 감소했습니다. 최근 생산량 증가와 새로운 화학물질 사용이 있었다면, 환경 엔지니어는 어떤 절차로 원인을 파악하고 개선 방안을 도출해야 할까요?",
            
            "인사 관리에서 한 부서의 이직률이 갑자기 높아졌는데, 최근 팀장이 바뀌었고, 업무량도 증가했습니다. 직원 만족도 조사에서는 소통 부족이 주요 문제로 지적되었습니다. HR 담당자는 이 상황을 어떻게 분석하고 조직 개선 방안을 단계적으로 실행해야 할까요?",
            
            "물류 관리에서 특정 지역으로의 배송이 자주 지연되고, 해당 지역 고객 불만이 증가했습니다. 날씨는 평년 수준이지만, 배송 기사 부족과 교통 체증이 심해졌다면, 물류 관리자는 어떤 방식으로 배송 최적화 방안을 찾고 실행해야 할까요?",
            
            "농업 컨설팅에서 농작물 수확량이 예년 대비 30% 감소했는데, 기후는 평년과 비슷했지만 토양 pH가 변했고 새로운 비료를 사용했습니다. 농업 전문가는 어떤 순서로 원인을 규명하고 내년도 개선 방안을 제시해야 할까요?",
            
            "제조업에서 제품 불량률이 갑자기 증가했는데, 새로운 원자재 공급업체를 사용하기 시작했고, 생산라인 운영자도 일부 교체되었습니다. 품질관리 담당자는 어떤 방법으로 불량 원인을 찾아내고 품질 개선 계획을 수립해야 할까요?",
            
            "부동산 컨설팅에서 특정 지역의 아파트 가격이 주변 지역과 다르게 움직이고 있습니다. 교통 인프라 개발 계획이 있고, 학군 변화도 예정되어 있다면, 부동산 전문가는 어떤 요소들을 종합적으로 분석해서 투자 가치를 평가해야 할까요?",
            
            "스포츠 의학에서 운동선수가 무릎 통증을 호소하는데, 최근 훈련 강도가 증가했고 체중도 늘었습니다. MRI 결과는 정상이지만 특정 동작에서만 통증이 발생합니다. 스포츠 의학 전문의는 어떤 접근 방식으로 원인을 찾고 치료 계획을 세워야 할까요?",
            
            "보험 심사에서 한 고객의 사고 청구가 들어왔는데, 사고 장소와 시간이 고객의 평소 이동 패턴과 다르고, 사고 차량의 손상 정도도 사고 경위와 일치하지 않는 부분이 있습니다. 보험 조사관은 어떤 방식으로 사실관계를 확인하고 결론을 도출해야 할까요?",
            
            "게임 운영에서 특정 서버의 동접자 수가 급격히 감소했는데, 새로운 업데이트 이후이고 커뮤니티에서는 밸런스 문제가 제기되고 있습니다. 게임 기획자는 어떤 데이터를 분석하고 어떤 순서로 문제를 해결해야 할까요?",
            
            "회계 감사에서 한 회사의 매출은 증가했지만 현금흐름이 악화되었고, 매출채권 회전율도 떨어졌습니다. 최근 대형 고객과의 계약 조건이 변경되었다면, 회계사는 어떤 관점에서 이 상황을 분석하고 경영진에게 어떤 조언을 해야 할까요?",
            
            "소셜미디어 관리에서 브랜드 언급량은 증가했지만 긍정적 반응은 감소했고, 특정 연령층에서만 부정적 반응이 집중되고 있습니다. 최근 브랜드 메시지를 변경했다면, 디지털 마케터는 어떤 방식으로 이 현상을 분석하고 대응 전략을 수립해야 할까요?"
        ]
        return questions
    
    def generate_response(self, question, max_length=1024, temperature=0.7) :
        try :
            prompt = f"질문 : {question}\n\n답변 : "

            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(self.device)
            
            #####################################
            # AutoModel
            with torch.no_grad() :
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_length,
                    temperature=temperature,
                    do_sample=True,
                    top_p=0.9,
                    top_k=50,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )

            generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = generated_text.replace(prompt, "").strip()
            print(f"답변 : {response}")
            #####################################

            return response
        
        except Exception as e :
            return f"답변 생성 중 오류 발생 : {e}"
        
    def run_test(self, output_dir="./result/6월2일") :
        os.makedirs(output_dir, exist_ok=True)

        if not self.load_model() :
            return False
        
        questions = self.get_test_questions()

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        result_file = os.path.join(output_dir, f"llm_test_results_{timestamp}.txt")

        all_results = []
        avg_time = 0
        with open(result_file, 'w', encoding="utf-8") as f :
            f.write(f"LLM 성능 테스트 결과\n")
            f.write(f"모델: {self.model_name}\n")
            f.write(f"테스트 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"총 질문 수: {len(questions)}\n")
            f.write("=" * 80 + "\n\n")

            for i, question in enumerate(questions, 1):
                print(f"\n[{i}/{len(questions)}] 질문 처리 중...")
                print(f"질문: {question[:100]}...")

                start_time = time.time()
                response = self.generate_response(question)
                end_time = time.time()
                elapsed_time = end_time - start_time
                print(f"처리 시간 : {elapsed_time:.2f}초")

                avg_time += end_time - start_time
                result = {
                    "question_num" : i,
                    "question" : question,
                    "response" : response,
                }

                all_results.append(result)

                f.write(f"질문 {i}:\n")
                f.write(f"{question}\n\n")
                f.write(f"답변:\n")
                f.write(f"{response}\n\n")
                f.write(f"처리 시간: {elapsed_time:.2f}초\n")
                f.write("-" * 80 + "\n\n")
                f.flush()
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
        avg_time = avg_time / len(questions)
        print(f"답변 평균 시간 : {avg_time:.2f}")
        return True
    

def main() :
    tester = LLMTester()

    success = tester.run_test()

    if success :
        print("테스트 완료")
    else :
        print("오류 발생")

if __name__ == "__main__" :
    main()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]


[1/20] 질문 처리 중...
질문: 한 의료진이 환자의 증상을 보고 '기침, 발열, 호흡곤란'이라고 기록했습니다. 이 환자가 최근 해외여행을 다녀왔고, 혈액검사에서 백혈구 수치가 높게 나왔다면, 어떤 추가 검사를 우...
답변 : 1) 환자 진료에 있어 기본적인 원칙 중 하나는 신속히 원인을 파악하고 적절한 치료를 시작하는 것입니다. 이 경우, 환자는 기침, 발열, 호흡곤란 등 다양한 증상이 나타나며 최근 해외 여행을 다녀왔습니다. 이는 심각한 감염병이 원인일 가능성이 있음을 시사합니다. 따라서, 우선 시행할 추가 검사는 다음과 같습니다:

- 가래 배양(sputum culture): 이는 폐렴이나 다른 하기도 감염의 흔한 원인인 세균을 식별하는 데 도움이 됩니다.

- 혈중 및 소변 내 세균 검사(blood and urine cultures): 신우신염과 같은 세균 감염 여부를 판단하기 위해 필요합니다.

- 항체 검사(antibody tests): 말라리아, 리케치아증, 일본뇌염과 같이 특정 해외 여행 지역과 연관된 질환들을 확인하는 데 중요합니다.

- 분자 검사(molecular testing): 결핵(TB) 또는 COVID-19과 같이 의심되는 질환을 신속하게 진단하는데 매우 유용하며, 전통적인 방법보다 민감도가 높습니다.

2) 추가 검사 외에도 감염관리 프로토콜을 준수하여 의료진과 환자를 보호해야 합니다. 다음 사항을 고려하세요:

- 개인보호장비(PPE) 사용: 의료진은 환자와 직접 접촉 전에 장갑, 가운, N95 마스크를 포함한 적절한 PPE를 착용해야 합니다.

- 공기전파 예방 조치: 환자와 의료진의 안전을 보장하기 위해 적절한 환기, 환기와 함께 음압실 사용, 비말 차단 마스크와 같은 조치를 취해야 합니다.

- 손 위생: 의료진은 정기적으로 손을 비누로 철저하게 씻거나 알코올 기반 손 소독제를 사용해야 합니다.

- 병원 내 전파 방지: 환자들은 그들의 방에서 격리되어 있어야 하며, 다른 환자들에게 병을 퍼뜨리지 않도록 해야 합니다

# T5

In [4]:
# from transformers import T5ForConditionalGeneration

# class LLMTester:
#     def __init__(self, model_name="./model/LLM/aya-101"):
#         self.model_name = model_name
#         self.device = "cuda" if torch.cuda.is_available() else "cpu"
#         self.tokenizer = None
#         self.model = None

#     def load_model(self) :
#         try :
#             quantization_config = BitsAndBytesConfig(
#                 load_in_4bit=True,
#                 bnb_4bit_compute_dtype=torch.float16,
#                 bnb_4bit_quant_type="nf4",
#                 bnb_4bit_use_double_quant=True
#             )

#             self.tokenizer = AutoTokenizer.from_pretrained(
#                 self.model_name,
#                 trust_remote_code=True
#             )

#             # device_map = "auto" if self.device=="cuda" else None
#             # device_map = {"": 2} if self.device == "cuda" else None

#             self.model = T5ForConditionalGeneration.from_pretrained(
#                 self.model_name,
#                 torch_dtype=torch.float16 if self.device=="cuda" else torch.float32,
#                 # device_map=device_map,
#                 trust_remote_code=True,
#                 low_cpu_mem_usage=True,
#                 quantization_config=quantization_config
#             )

#             if self.tokenizer.pad_token is None :
#                 self.tokenizer.pad_token = self.tokenizer.eos_token

#             return True

#         except Exception as e :
#             print(f"오류 발생 : {e}")
#             return False
        
#     def get_test_questions(self) :
#         questions = [
#             "한 의료진이 환자의 증상을 보고 '기침, 발열, 호흡곤란'이라고 기록했습니다. 이 환자가 최근 해외여행을 다녀왔고, 혈액검사에서 백혈구 수치가 높게 나왔다면, 어떤 추가 검사를 우선적으로 실시해야 하며, 그 이유는 무엇인가요? 또한 이 과정에서 의료진이 고려해야 할 감염관리 프로토콜은 무엇인가요?",
            
#             "금융 시장에서 A 기업의 주가가 급락하고, 같은 업종의 B, C 기업 주가도 동반 하락했습니다. 그런데 D 기업만 상승했다면, 이런 현상의 원인을 어떻게 분석해야 하며, 투자자는 어떤 정보를 추가로 수집해서 투자 판단을 내려야 할까요?",
            
#             "법률 상담에서 임차인이 '집주인이 갑자기 전세 계약 해지를 통보했고, 이사할 돈이 없다'고 상담을 요청했습니다. 이때 변호사는 어떤 순서로 사실관계를 파악해야 하며, 임차인의 권리를 보호하기 위해 어떤 법적 조치들을 단계적으로 검토해야 할까요?",
            
#             "교육 현장에서 한 학생이 수학 성적은 우수하지만 국어 성적이 급격히 떨어졌고, 최근 수업 참여도도 낮아졌습니다. 학부모는 사교육을 늘리자고 하는데, 교사는 어떤 관점에서 이 상황을 분석하고 어떤 해결책을 단계적으로 제시해야 할까요?",
            
#             "IT 시스템에서 특정 서버의 CPU 사용률이 90%를 넘고, 응답 시간이 평소보다 3배 느려졌습니다. 동시에 데이터베이스 연결 수도 임계치에 도달했다면, 시스템 관리자는 어떤 순서로 문제를 진단하고 해결해야 하며, 근본적인 원인을 찾기 위해 어떤 로그들을 분석해야 할까요?",
            
#             "마케팅 담당자가 온라인 광고 캠페인을 분석하니 클릭률은 높은데 구매 전환율이 낮습니다. 그런데 모바일에서는 전환율이 더 낮고, 특정 연령대에서만 이런 현상이 두드러집니다. 이 데이터를 바탕으로 어떤 가설을 세우고 어떤 개선 방안을 단계적으로 실행해야 할까요?",
            
#             "건축 현장에서 콘크리트 타설 후 균열이 발견되었고, 날씨는 평소보다 추웠으며, 사용된 콘크리트의 배합비가 기존과 달랐습니다. 구조 엔지니어는 어떤 방식으로 이 문제의 원인을 규명하고, 안전성을 확보하기 위해 어떤 조치들을 순차적으로 취해야 할까요?",
            
#             "심리 상담에서 내담자가 '직장에서 스트레스를 받아 잠을 못 자고, 가족과도 자주 다툰다'고 호소합니다. 그런데 대화 중 어린 시절 이야기를 할 때 표정이 어두워집니다. 상담사는 어떤 방식으로 이 정보들을 연결하여 상담 방향을 설정하고 개입 전략을 수립해야 할까요?",
            
#             "레스토랑 운영에서 최근 매출이 20% 감소했는데, 근처에 새로운 경쟁업체가 생겼고, 주요 메뉴의 원재료 가격도 올랐습니다. 고객 리뷰에서는 서비스 지연에 대한 불만이 증가했습니다. 경영자는 이 복합적인 문제를 어떻게 분석하고 우선순위를 정해 해결책을 실행해야 할까요?",
            
#             "환경 컨설팅에서 한 공장의 폐수 처리 시설에서 처리 효율이 갑자기 떨어졌고, 미생물 활성도도 감소했습니다. 최근 생산량 증가와 새로운 화학물질 사용이 있었다면, 환경 엔지니어는 어떤 절차로 원인을 파악하고 개선 방안을 도출해야 할까요?",
            
#             "인사 관리에서 한 부서의 이직률이 갑자기 높아졌는데, 최근 팀장이 바뀌었고, 업무량도 증가했습니다. 직원 만족도 조사에서는 소통 부족이 주요 문제로 지적되었습니다. HR 담당자는 이 상황을 어떻게 분석하고 조직 개선 방안을 단계적으로 실행해야 할까요?",
            
#             "물류 관리에서 특정 지역으로의 배송이 자주 지연되고, 해당 지역 고객 불만이 증가했습니다. 날씨는 평년 수준이지만, 배송 기사 부족과 교통 체증이 심해졌다면, 물류 관리자는 어떤 방식으로 배송 최적화 방안을 찾고 실행해야 할까요?",
            
#             "농업 컨설팅에서 농작물 수확량이 예년 대비 30% 감소했는데, 기후는 평년과 비슷했지만 토양 pH가 변했고 새로운 비료를 사용했습니다. 농업 전문가는 어떤 순서로 원인을 규명하고 내년도 개선 방안을 제시해야 할까요?",
            
#             "제조업에서 제품 불량률이 갑자기 증가했는데, 새로운 원자재 공급업체를 사용하기 시작했고, 생산라인 운영자도 일부 교체되었습니다. 품질관리 담당자는 어떤 방법으로 불량 원인을 찾아내고 품질 개선 계획을 수립해야 할까요?",
            
#             "부동산 컨설팅에서 특정 지역의 아파트 가격이 주변 지역과 다르게 움직이고 있습니다. 교통 인프라 개발 계획이 있고, 학군 변화도 예정되어 있다면, 부동산 전문가는 어떤 요소들을 종합적으로 분석해서 투자 가치를 평가해야 할까요?",
            
#             "스포츠 의학에서 운동선수가 무릎 통증을 호소하는데, 최근 훈련 강도가 증가했고 체중도 늘었습니다. MRI 결과는 정상이지만 특정 동작에서만 통증이 발생합니다. 스포츠 의학 전문의는 어떤 접근 방식으로 원인을 찾고 치료 계획을 세워야 할까요?",
            
#             "보험 심사에서 한 고객의 사고 청구가 들어왔는데, 사고 장소와 시간이 고객의 평소 이동 패턴과 다르고, 사고 차량의 손상 정도도 사고 경위와 일치하지 않는 부분이 있습니다. 보험 조사관은 어떤 방식으로 사실관계를 확인하고 결론을 도출해야 할까요?",
            
#             "게임 운영에서 특정 서버의 동접자 수가 급격히 감소했는데, 새로운 업데이트 이후이고 커뮤니티에서는 밸런스 문제가 제기되고 있습니다. 게임 기획자는 어떤 데이터를 분석하고 어떤 순서로 문제를 해결해야 할까요?",
            
#             "회계 감사에서 한 회사의 매출은 증가했지만 현금흐름이 악화되었고, 매출채권 회전율도 떨어졌습니다. 최근 대형 고객과의 계약 조건이 변경되었다면, 회계사는 어떤 관점에서 이 상황을 분석하고 경영진에게 어떤 조언을 해야 할까요?",
            
#             "소셜미디어 관리에서 브랜드 언급량은 증가했지만 긍정적 반응은 감소했고, 특정 연령층에서만 부정적 반응이 집중되고 있습니다. 최근 브랜드 메시지를 변경했다면, 디지털 마케터는 어떤 방식으로 이 현상을 분석하고 대응 전략을 수립해야 할까요?"
#         ]
#         return questions
    
#     def generate_response(self, question, max_length=1024, temperature=0.7) :
#         try :
#             prompt = f"질문 : {question}\n\n답변 : "

#             inputs = self.tokenizer(
#                 prompt,
#                 return_tensors="pt",
#                 padding=True,
#                 truncation=True,
#                 max_length=512
#             ).to(self.device)
            
#             #####################################
#             # AutoModel
#             with torch.no_grad() :
#                 outputs = self.model.generate(
#                     **inputs,
#                     max_length=max_length,
#                     temperature=temperature,
#                     do_sample=True,
#                     top_p=0.9,
#                     top_k=50,
#                     pad_token_id=self.tokenizer.eos_token_id,
#                     eos_token_id=self.tokenizer.eos_token_id,
#                     repetition_penalty=1.1
#                 )

#             generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
#             response = generated_text.replace(prompt, "").strip()
#             #####################################

#             return response
        
#         except Exception as e :
#             return f"답변 생성 중 오류 발생 : {e}"
        
#     def run_test(self, output_dir="./result/6월2일") :
#         os.makedirs(output_dir, exist_ok=True)

#         if not self.load_model() :
#             return False
        
#         questions = self.get_test_questions()

#         timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#         result_file = os.path.join(output_dir, f"llm_test_results_{timestamp}.txt")

#         all_results = []
#         avg_time = 0
#         with open(result_file, 'w', encoding="utf-8") as f :
#             f.write(f"LLM 성능 테스트 결과\n")
#             f.write(f"모델: {self.model_name}\n")
#             f.write(f"테스트 시간: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
#             f.write(f"총 질문 수: {len(questions)}\n")
#             f.write("=" * 80 + "\n\n")

#             for i, question in enumerate(questions, 1):
#                 print(f"\n[{i}/{len(questions)}] 질문 처리 중...")
#                 print(f"질문: {question[:100]}...")

#                 start_time = time.time()
#                 response = self.generate_response(question)
#                 end_time = time.time()
#                 elapsed_time = end_time - start_time
#                 print(f"처리 시간 : {elapsed_time:.2f}초")

#                 avg_time += end_time - start_time
#                 result = {
#                     "question_num" : i,
#                     "question" : question,
#                     "response" : response,
#                 }

#                 all_results.append(result)

#                 f.write(f"질문 {i}:\n")
#                 f.write(f"{question}\n\n")
#                 f.write(f"답변:\n")
#                 f.write(f"{response}\n\n")
#                 f.write(f"처리 시간: {elapsed_time:.2f}초\n")
#                 f.write("-" * 80 + "\n\n")
#                 f.flush()
#                 torch.cuda.empty_cache() if torch.cuda.is_available() else None
#         avg_time = avg_time / len(questions)
#         print(f"답변 평균 시간 : {avg_time:.2f}")
#         return True
    

# def main() :
#     tester = LLMTester()

#     success = tester.run_test()

#     if success :
#         print("테스트 완료")
#     else :
#         print("오류 발생")

# if __name__ == "__main__" :
#     main()

# 역번역

In [5]:
import random
import time
import json
from typing import List, Dict, Optional
from translate import *

class BackTranslationAugmentation:
    def __init__(self, model=None, tokenizer=None):
        self.model = model
        self.tokenizer = tokenizer
        
    def back_translate_ko_en_ko(self, text: str) -> str:
        """
        한국어 -> 영어 -> 한국어 역번역
        """
        try:
            english_text = ko_to_en(text, self.model, self.tokenizer)
            print(f"원문: {text}")
            print(f"영어 번역: {english_text}")
            
            back_translated = en_to_ko(english_text, self.model, self.tokenizer)
            print(f"역번역: {back_translated}")
            print("-" * 50)
            
            return back_translated
            
        except Exception as e:
            print(f"역번역 중 오류 발생: {e}")
            return text  
    
    def back_translate_en_ko_en(self, text: str) -> str:
        """
        영어 -> 한국어 -> 영어 역번역
        """
        try:
            korean_text = en_to_ko(text, self.model, self.tokenizer)
            print(f"원문: {text}")
            print(f"한국어 번역: {korean_text}")
            
            back_translated = ko_to_en(korean_text, self.model, self.tokenizer)
            print(f"역번역: {back_translated}")
            print("-" * 50)
            
            return back_translated
            
        except Exception as e:
            print(f"역번역 중 오류 발생: {e}")
            return text  
    
    def augment_text_list(self, texts: List[str], language: str = "ko", 
                         augment_ratio: float = 1.0) -> List[str]:
        """
        텍스트 리스트를 역번역으로 증강
        
        Args:
            texts: 원본 텍스트 리스트
            language: 원본 언어 ("ko" 또는 "en")
            augment_ratio: 증강 비율 (1.0 = 100% 증강, 0.5 = 50% 증강)
        
        Returns:
            원본 + 증강된 텍스트 리스트
        """
        augmented_texts = texts.copy()  

        num_to_augment = int(len(texts) * augment_ratio)
        texts_to_augment = random.sample(texts, num_to_augment)
        
        print(f"총 {len(texts)}개 텍스트 중 {num_to_augment}개를 증강합니다.")
        
        for i, text in enumerate(texts_to_augment, 1):
            print(f"\n[{i}/{num_to_augment}] 처리 중...")
            
            if language == "ko":
                augmented = self.back_translate_ko_en_ko(text)
            elif language == "en":
                augmented = self.back_translate_en_ko_en(text)
            else:
                print(f"지원하지 않는 언어: {language}")
                continue
            
            if augmented != text and augmented.strip():
                augmented_texts.append(augmented)
            
            time.sleep(0.5)
        
        print(f"\n원본: {len(texts)}개 -> 증강 후: {len(augmented_texts)}개")
        return augmented_texts
    
    def augment_dataset(self, dataset: List[Dict], text_field: str, 
                       language: str = "ko", augment_ratio: float = 1.0) -> List[Dict]:
        """
        데이터셋의 특정 필드를 역번역으로 증강
        
        Args:
            dataset: 딕셔너리 리스트 형태의 데이터셋
            text_field: 증강할 텍스트 필드명
            language: 원본 언어
            augment_ratio: 증강 비율
        
        Returns:
            증강된 데이터셋
        """
        augmented_dataset = dataset.copy()
        
        num_to_augment = int(len(dataset) * augment_ratio)
        data_to_augment = random.sample(dataset, num_to_augment)
        
        print(f"데이터셋 증강: {len(dataset)}개 -> 예상 {len(dataset) + num_to_augment}개")
        
        for i, data in enumerate(data_to_augment, 1):
            print(f"\n[{i}/{num_to_augment}] 데이터 처리 중...")
            
            original_text = data[text_field]
            
            if language == "ko":
                augmented_text = self.back_translate_ko_en_ko(original_text)
            elif language == "en":
                augmented_text = self.back_translate_en_ko_en(original_text)
            else:
                continue
            
            if augmented_text != original_text and augmented_text.strip():
                augmented_data = data.copy()
                augmented_data[text_field] = augmented_text
                augmented_data['augmented'] = True  
                augmented_dataset.append(augmented_data)
            
            time.sleep(0.5)
        
        return augmented_dataset
    
    def save_augmented_data(self, data: List, filename: str, format: str = "json"):
        """
        증강된 데이터를 파일로 저장
        """
        if format == "json":
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
        elif format == "txt":
            with open(filename, 'w', encoding='utf-8') as f:
                for item in data:
                    if isinstance(item, str):
                        f.write(item + '\n')
                    else:
                        f.write(str(item) + '\n')
        
        print(f"데이터가 {filename}에 저장되었습니다.")

def example_usage():
    """
    역번역 데이터 증강 사용 예시
    """
    augmenter = BackTranslationAugmentation()
    
    korean_texts = [
        "안녕하세요. 좋은 하루 되세요.",
        "오늘 날씨가 정말 좋네요.",
        "파이썬 프로그래밍을 배우고 있습니다.",
        "머신러닝은 흥미로운 분야입니다."
    ]
    
    print("=== 텍스트 리스트 증강 ===")
    augmented_texts = augmenter.augment_text_list(
        korean_texts, 
        language="ko", 
        augment_ratio=0.5  
    )
    
    augmenter.save_augmented_data(augmented_texts, "augmented_texts.txt", "txt")
    
    qa_dataset = [
        {
            "question": "파이썬이란 무엇인가요?",
            "answer": "파이썬은 간단하고 읽기 쉬운 프로그래밍 언어입니다.",
            "category": "programming"
        },
        {
            "question": "머신러닝의 종류는?",
            "answer": "지도학습, 비지도학습, 강화학습으로 나뉩니다.",
            "category": "ml"
        }
    ]
    
    print("\n=== 데이터셋 증강 ===")
    augmented_dataset = augmenter.augment_dataset(
        qa_dataset, 
        text_field="question", 
        language="ko",
        augment_ratio=1.0  
    )
    
    augmented_dataset = augmenter.augment_dataset(
        augmented_dataset,
        text_field="answer",
        language="ko", 
        augment_ratio=0.5  
    )
    
    augmenter.save_augmented_data(augmented_dataset, "augmented_qa_dataset.json", "json")

def batch_augmentation(input_file: str, output_file: str, 
                      text_field: str = None, language: str = "ko"):
    """
    파일에서 데이터를 읽어서 배치로 증강
    """
    augmenter = BackTranslationAugmentation()
    
    # 파일 읽기
    if input_file.endswith('.json'):
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if text_field:  
            augmented = augmenter.augment_dataset(data, text_field, language)
        else:  
            augmented = augmenter.augment_text_list(data, language)
            
    elif input_file.endswith('.txt'):
        with open(input_file, 'r', encoding='utf-8') as f:
            data = [line.strip() for line in f if line.strip()]
        
        augmented = augmenter.augment_text_list(data, language)
    
    augmenter.save_augmented_data(augmented, output_file)
    print(f"배치 증강 완료: {input_file} -> {output_file}")

if __name__ == "__main__":
    example_usage()
    
    # batch_augmentation("input_data.json", "output_data.json", "text", "ko")

=== 텍스트 리스트 증강 ===
총 4개 텍스트 중 2개를 증강합니다.

[1/2] 처리 중...
구두점으로 분할된 문장 리스트 : ['오늘 날씨가 정말 좋네요.']
원문: 오늘 날씨가 정말 좋네요.
영어 번역: The weather is really nice today.
구두점으로 분할된 문장 리스트 : ['The weather is really nice today.']
역번역: 오늘 날씨가 정말 좋아요.
--------------------------------------------------

[2/2] 처리 중...
구두점으로 분할된 문장 리스트 : ['머신러닝은 흥미로운 분야입니다.']
원문: 머신러닝은 흥미로운 분야입니다.
영어 번역: Machine learning is an exciting field.
구두점으로 분할된 문장 리스트 : ['Machine learning is an exciting field.']
역번역: 기계 학습은 흥미로운 분야입니다.
--------------------------------------------------

원본: 4개 -> 증강 후: 6개
데이터가 augmented_texts.txt에 저장되었습니다.

=== 데이터셋 증강 ===
데이터셋 증강: 2개 -> 예상 4개

[1/2] 데이터 처리 중...
구두점으로 분할된 문장 리스트 : ['파이썬이란 무엇인가요?']


Some parameters are on the meta device because they were offloaded to the cpu.


원문: 파이썬이란 무엇인가요?
영어 번역: What is Python?
구두점으로 분할된 문장 리스트 : ['What is Python?']
역번역: 파이썬이란 무엇인가?
--------------------------------------------------

[2/2] 데이터 처리 중...
구두점으로 분할된 문장 리스트 : ['머신러닝의 종류는?']
원문: 머신러닝의 종류는?
영어 번역: What are the types of machine learning?
구두점으로 분할된 문장 리스트 : ['What are the types of machine learning?']


Some parameters are on the meta device because they were offloaded to the cpu.


역번역: 기계 학습의 유형은 무엇입니까?
--------------------------------------------------
데이터셋 증강: 4개 -> 예상 6개

[1/2] 데이터 처리 중...
구두점으로 분할된 문장 리스트 : ['파이썬은 간단하고 읽기 쉬운 프로그래밍 언어입니다.']
원문: 파이썬은 간단하고 읽기 쉬운 프로그래밍 언어입니다.
영어 번역: Python is a simple and easy-to-read programming language.
구두점으로 분할된 문장 리스트 : ['Python is a simple and easy-to-read programming language.']
역번역: 파이썬은 간단하고 읽기 쉬운 프로그래밍 언어입니다.
--------------------------------------------------

[2/2] 데이터 처리 중...
구두점으로 분할된 문장 리스트 : ['지도학습, 비지도학습, 강화학습으로 나뉩니다.']


Some parameters are on the meta device because they were offloaded to the cpu.


원문: 지도학습, 비지도학습, 강화학습으로 나뉩니다.
영어 번역: It is divided into guided learning, non-guided learning and reinforcement learning.
구두점으로 분할된 문장 리스트 : ['It is divided into guided learning, non-guided learning and reinforcement learning.']


Some parameters are on the meta device because they were offloaded to the cpu.


역번역: 가이드 학습, 비가이드학습 및 강화학습으로 나뉜다.
--------------------------------------------------
데이터가 augmented_qa_dataset.json에 저장되었습니다.
